In [ ]:
import pandas as pd
import polars as pl

df = pd.read_csv("/Users/tweber/Data/mosaicatcher_samples_metadata.csv", sep=";")
dt = pl.read_delta(
    "/Users/tweber/Gits/depictio/minio_data/depictio-bucket/65e5f007bad32df857a53cf2/65fc33d505358fc517a6e78c/65fc33d505358fc517a6e78d"
)
dt = dt.to_pandas()
dt[["depictio_run_id", "sample"]].set_index(
    "depictio_run_id"
).drop_duplicates().to_records().tolist()

In [ ]:
l = list()
df_copy = df.copy()
for e in (
    dt[["depictio_run_id", "sample"]]
    .set_index("depictio_run_id")
    .drop_duplicates()
    .to_records()
    .tolist()
):
    tmp_df = pd.read_csv(
        f"/Users/tweber/Data/mosaicatcher-pipeline/{e[0]}/{e[1]}/cell_selection/labels.tsv",
        sep="\t",
    )
    l.append(tmp_df)
l = pd.concat(l)
pd.merge(df_copy, l[["sample", "cell"]], left_on="sample", right_on="sample").to_csv(
    "/Users/tweber/Data/mosaicatcher-pipeline/mosaicatcher_samples_metadata.csv",
    sep=";",
    index=False,
)

In [ ]:
# from gridfs import GridFS
import pymongo


MONGODB_URL = f"mongodb://0.0.0.0:27018/"
print("MONGODB_URL: ", MONGODB_URL)


client = pymongo.MongoClient(MONGODB_URL)
db = client["depictioDB"]


data_collection = db["data_collections"]
workflow_collection = db["workflows"]
runs_collection = db["runs"]
files_collection = db["files"]
users_collection = db["users"]
projects_collection = db["projects"]
# groups_collection = db["groups_collection"]

In [ ]:
from bson import ObjectId

target_data_collection_id = ObjectId("67a25c82fea6466823de362f")


# Query to find any document containing the specified data collection ID
query = {"workflows.data_collections._id": target_data_collection_id}

# Execute the search
result = projects_collection.find(query)

# Print the matching documents
for doc in result:
    print(doc)

In [ ]:
user_oid = ObjectId("67658ba033c8b59ad489d7c7")
# Aggregation pipeline
pipeline = [
    {
        "$match": {
            "$or": [
                {"permissions.owners._id": user_oid},  # User is an owner
                {"permissions.owners.is_admin": True},  # User is an admin
            ]
        }
    },
    {"$unwind": "$workflows"},
    {"$unwind": "$workflows.data_collections"},
    {
        "$match": {"workflows.data_collections._id": target_data_collection_id}
    },  # Find specific data collection
    {
        "$project": {"_id": 0, "data_collection": "$workflows.data_collections"}
    },  # Extract only the data_collection
]

# Execute the aggregation
result = projects_collection.aggregate(pipeline)

# Print matching data collections
for doc in result:
    print(doc["data_collection"])

In [ ]:
# Query in files_collection
files_collection.find_one(
    {"file_location": "/Users/tweber/Data/mosaicatcher-pipeline/mosaicatcher_samples_metadata.csv"}
)
# Show all documents from workflow_collection
# for doc in workflow_collection.find():
#     print(doc)

# remove that file from files_collection
files_collection.delete_one(
    {"file_location": "/Users/tweber/Data/mosaicatcher-pipeline/mosaicatcher_samples_metadata.csv"}
)

In [ ]:
# The ID of the data collection you're interested in
from bson import ObjectId


data_collection_id = ObjectId("6647d03b57d1fae8d89dce90")

# Query to retrieve all files belonging to the specified data collection
files = files_collection.find({"data_collection._id": data_collection_id})
for file in files[:10]:
    print(file)
    if (
        file["file_location"]
        == "/Users/tweber/Data/mosaicatcher-pipeline/mosaicatcher_samples_metadata.csv.bak"
    ):
        print("Found")
        # delete the file
        files_collection.delete_one({"_id": file["_id"]})

In [ ]:
# Constructing the nested dictionary
import collections
from pprint import pprint

nested_dict = collections.defaultdict(lambda: collections.defaultdict(dict))

data_collection_id = ObjectId("6647d03b57d1fae8d89dce90")
files = files_collection.find({"data_collection._id": data_collection_id})
for file in files:
    # print(file)
    if file["filename"].endswith(
        file["data_collection"]["config"]["dc_specific_properties"]["index_extension"]
    ):
        continue
    for wildcard in file["wildcards"]:
        if file["trackId"]:
            nested_dict[file["data_collection"]["_id"]][wildcard["name"]][wildcard["value"]] = file[
                "trackId"
            ]

# Example output
pprint(nested_dict)

In [ ]:
# id=ObjectId('65fb5bb927b90358513efd08') file_location=PosixPath('/Users/tweber/Data/mosaicatcher-pipeline/2019-09-11-HTN7CAFXY/HG01114x02/plots/JBROWSE/HG01114x02PE20356-SV.bed.gz.tbi') S3_location=None filename='HG01114x02PE20356-SV.bed.gz.tbi' creation_time=datetime.datetime(2024, 1, 29, 10, 5, 44) modification_time=datetime.datetime(2024, 1, 24, 6, 1, 31) data_collection=DataCollection(id=ObjectId('65fb5bb927b90358513efd06'), data_collection_tag='SV_calls', description='BED files containing SV calls generated by MosaiCatcher', config=DataCollectionConfig(type='JBrowse2', regex=Regex(pattern='{cell}-SV.bed.gz', type='file-based', wildcards=[WildcardRegexBase(name='cell', wildcard_regex='.*')]), dc_specific_properties=DCJBrowse2Config(index_extension='tbi', jbrowse_template_location='/Users/tweber/Gits/depictio/CLI_client/configs/mosaicatcher_pipeline/jbrowse2_templates/SV_calls.json'), join=TableJoinConfig(on_columns=['cell'], how='inner', with_dc=['mosaicatcher_stats']))) run_id='2019-09-11-HTN7CAFXY' registration_time=datetime.datetime(2024, 3, 20, 21, 57, 7, 395615) wildcards=[WildcardRegex(name='cell', wildcard_regex='.*', value='HG01114x02PE20356')]
# Find all files with a specific data_collection_id
for file in files_collection.find({"data_collection": {"_id": "660c1dabf22783012cc91384"}}):
    print(file)

In [ ]:
import httpx

API_BASE_URL = "http://0.0.0.0:8058"
last_jbrowse_status = httpx.get(f"{API_BASE_URL}/depictio/api/v1/jbrowse/last_status")
last_jbrowse_status = last_jbrowse_status.json()
last_jbrowse_status